In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ta

In [ ]:
dataset_train = pd.read_csv('train.csv')
dataset_train = dataset_train.drop(["Date","Turnover (Lacs)"], axis=1)
RSI=ta.momentum.rsi(close=dataset_train['Close'])
MFI=ta.volume.money_flow_index(dataset_train['High'], dataset_train['Low'], dataset_train['Close'], dataset_train['Total Trade Quantity'],n=14)
dataset_train["RSI"]= RSI
dataset_train["MFI"]=MFI
dataset_train= ta.utils.dropna(dataset_train)
dataset_train.head()

In [ ]:
training_set = dataset_train.iloc[:,:].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
X_train = []
y_train = []
for i in range(60, 2000):
    X_train.append(training_set_scaled[i-60:i])
    y_train.append(training_set_scaled[i,4])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 8))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[1], 8)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 60))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
dataset_test = pd.read_csv('test.csv')
real_stock_price = dataset_test.iloc[:, 5:6].values
dataset_test = dataset_test.drop(["Date","Turnover (Lacs)"], axis=1)
RSI=ta.momentum.rsi(close=dataset_test['Close'])
MFI=ta.volume.money_flow_index(dataset_test['High'], dataset_test['Low'], dataset_test['Close'], dataset_test['Total Trade Quantity'],n=14)
dataset_test["RSI"]= RSI
dataset_test["MFI"]=MFI
dataset_test= ta.utils.dropna(dataset_test)

In [ ]:
dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,8)
inputs = sc.transform(inputs)

In [ ]:
X_test = []
for i in range(60, 1000) :
    X_test.append(inputs[i-60:i])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],8))
predicted_stock_price = regressor.predict(X_test)

In [ ]:
predicted_stock_price1 = np.zeros(shape=(len(X_test), 8) )
predicted_stock_price1[:,4] = predicted_stock_price[:,0]
predicted_stock_price2 = sc.inverse_transform(predicted_stock_price1)[:,4]

In [ ]:
plt.plot(real_stock_price, color = 'black', label = 'stock Prices')
plt.plot(predicted_stock_price2, color = 'red', label = 'National stock Price Prediction')
plt.title('Predicted National Stock Price')
plt.xlabel('Time')
plt.ylabel( 'Stock Prices')
plt.legend()
plt.show()

In [ ]:
efficiency_percentage=((predicted_stock_price2/real_stock_price)*100)
efficiency_percentage.mean()